In [ ]:
#CODE MADE FOR BOTH LEPHARE SELECTED CANDIDATES AND COLOUR SELECTION SO FILES MIGHT HAVE DIFFERENT FORMATS
import astropy 
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import astropy_mpl_style
import os
import ccdproc as ccdp
from astropy.nddata import CCDData
from pathlib import Path
from os.path import isfile, join


In [ ]:
from matplotlib.colors import Normalize
import numpy as np
import re

In [ ]:
file = open('dual_image_catalogues/SED_SELECTED_DET277W','r')

x = []
y = []
xmin = []
xmax=[]
ymin=[]
ymax=[]
index = []
lines = []

#for _ in range(1): 
  #  next(file)
for line in file: 
    tokens = line.split(',')
    index.append(tokens[0])
    x.append(float(tokens[1]))
    y.append(float(tokens[2]))
file.close()

for i in range(len(index)):
    xmin.append(x[i]-50)
    xmax.append(x[i]+50)
    ymin.append(y[i]-50)
    ymax.append(y[i]+50)


In [ ]:
#making directory path to place files so they are organised in folders with index
for i in range(len(xmax)):
    os.makedirs(os.path.join('galaxy_samples/detf277w_SED_selection/' + str(index[i])))

In [ ]:
#making cut out of each image for each source in each filter
filts = ['f444w','f356w','f277w','f200w','f150w','f115w','f090w']
for j in filts: 
    uri = 'jwst_glass_combine_dv1_'+j+'_30mas_microJy.fits'

    for i in range(len(xmax)):
        with fits.open(uri) as hdul: 
            if int(xmin[i])<0: # just in case source is right at edge of image to save notebook from giving up
                xmin[i]==0
            elif int(ymin[i])<0:
                ymin[i]==0

            cutout = hdul[0].section[int(ymin[i]):int(ymax[i]),int(xmin[i]):int(xmax[i])]

            hdul.close()
            hdu = fits.PrimaryHDU(cutout)
            hdul = fits.HDUList([hdu]) #outputs postage stamp as fits file

            hdul.writeto('galaxy_samples/detf277w_SED_selection/'+str(index[i])+'/cutout'+j+'.fits',overwrite=True)


In [ ]:
indexcs = [] #making list of indexes to loop over

file = open('dual_image_catalogues/SED_SELECTED_DET277W','r')
#for _ in range(1):
 #   next(file)
for line in file:
    test = re.sub("\s+", ",", line.strip())
    tokens= test.split(',')
   
    indexcs.append(tokens[0])


In [ ]:
for i in range(len(indexcs)): #kernel sometimes times out so it is useful to know the value it stopped at
    if indexcs[i]== '15774':
        print(i)
print(len(indexcs))

In [ ]:
for i in indexcs:
    filenames = np.array(['galaxy_samples/detf277w_SED_selection/'+str(i)+'/'+f for f in os.listdir('galaxy_samples/detf277w_SED_selection/'+str(i)+'/') if isfile(join('galaxy_samples/detf277w_SED_selection/'+str(i)+'/',f))])
#makes array of filenames based on directory created above for different indexes i.e reads in filenames for each index
   
    f090w=CCDData.read(filenames[6],unit = 'adu') # read in strange order for some reason - taken account of
    f115w = CCDData.read(filenames[5],unit = 'adu')
    f150w =CCDData.read(filenames[2],unit = 'adu')
    f200w =CCDData.read(filenames[3],unit = 'adu')
    f277w =CCDData.read(filenames[1],unit = 'adu')
    f356w =CCDData.read(filenames[0],unit = 'adu')
    f444w =CCDData.read(filenames[4],unit = 'adu')

    fig = plt.figure()
    axes = fig.subplots(nrows=1, ncols=7)
    fig.set_figheight(100)
    fig.set_figwidth(100)

    #making postage stamps in a row for easier spotting of dropouts 
    #vmax acts as autocut in gaia, Normalize used to make images have sensible constrast
    axes[0].imshow(f090w,cmap = 'Greys', norm = Normalize(vmin = np.percentile(f090w,1.),vmax=np.percentile(f090w,98)))
    axes[1].imshow(f115w,cmap = 'Greys', norm = Normalize(vmin = np.percentile(f115w,1.),vmax=np.percentile(f115w,98)))
    axes[2].imshow(f150w,cmap = 'Greys', norm = Normalize(vmin = np.percentile(f150w,1.),vmax=np.percentile(f150w,98)))
    axes[3].imshow(f200w, cmap = 'Greys', norm = Normalize(vmin = np.percentile(f200w,1.),vmax=np.percentile(f200w,98)))
    axes[4].imshow(f277w, cmap = 'Greys', norm = Normalize(vmin = np.percentile(f277w,1.),vmax=np.percentile(f277w,98)))
    axes[5].imshow(f356w,cmap = 'Greys', norm = Normalize(vmin = np.percentile(f356w,1.),vmax=np.percentile(f356w,98)))
    axes[6].imshow(f444w,cmap = 'Greys', norm = Normalize(vmin = np.percentile(f444w,1.),vmax=np.percentile(f444w,98)))

    plt.savefig('galaxy_samples/detf277w_SED_selection/images/index'+i+'.png')
#saves 7 postage stamps as one figure in new folder to make it easier to view